In [17]:
# importing necessary packages

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

import time
import pprint
import pymongo
from pymongo import MongoClient
import pandas as pd
import tqdm

import psycopg2
from sqlalchemy import create_engine 

In [18]:
# defining a universal pause time
SCROLL_PAUSE_TIME = 1.5

In [34]:
# creating a connection
conn = psycopg2.connect(
        database = 'tfl_database',
        user = 'postgres',
        password = 'Hemanthkumar#1',
        host = 'localhost',
        port = '5432'
)

# creating a cursor object
cursor = conn.cursor()

In [20]:
# fetching the table
cursor.execute('SELECT * FROM underground_stations')

result = cursor.fetchall()

# converting the table to a pandas dataframe
stations_df = pd.DataFrame(result)
stations_df.columns = ['station_id', 'station', 'lines', 'local_authority', 'fare_zone', 'opened_on', 'annual_usage', 'area_served', 'url']
stations_df.set_index('station_id', inplace=True)
stations_df.head()

,station,lines,local_authority,fare_zone,opened_on,annual_usage,area_served,url
station_id,,,,,,,,
0,Acton Town,DP,Ealing,3,01-07-1879,4.93,Acton,https://www.google.com/maps?q=Acton+Town+Under...
2,Aldgate East,DH,Tower Hamlets,1,06-10-1884,10.23,Whitechapel,https://www.google.com/maps?q=Aldgate+East+Und...
3,Alperton,P,Brent,4,28-06-1903,2.30,Alperton,https://www.google.com/maps?q=Alperton+Undergr...
4,Amersham,M,Bucking­ham­shire,9,01-09-1892,1.66,Amersham,https://www.google.com/maps?q=Amersham+Undergr...
5,Angel,N,Islington,1,17-11-1901,12.37,Angel,https://www.google.com/maps?q=Angel+Undergroun...


In [5]:
if False:
    stations_df.to_csv('C:/Users/snkri/OneDrive/Desktop/tfl_project/stations_df.csv')

In [6]:
# reading the correct stations urls dataframe

good_urls = pd.read_csv("C:/Users/snkri/OneDrive/Desktop/tfl_project/good_stations.csv", index_col = [0])

good_urls.head()

,station,lines,url
index,,,
1,Aldgate,OM,https://www.google.com/maps/place/Aldgate/@51....
13,Barking,DH,https://www.google.com/maps/place/Barking/@51....
22,Blackfriars,OD,https://www.google.com/maps/place/Blackfriars/...
41,Chalfont & Latimer,M,https://www.google.com/maps/place/Chalfont+%26...
51,Clapham South,N,https://www.google.com/maps/place/Clapham+Sout...


In [7]:
j = 0

for i in good_urls.index:
        #print("____test____")
        url = good_urls.at[i, 'url']
        if "'" in url:
                url = url.replace("'", "''")
                sql = 'UPDATE underground_stations SET url = ' + "'" + url + "'" + " WHERE station_id = " + str(i) + ";"
        else:
                sql = 'UPDATE underground_stations SET url = ' + "'" + url + "'" + " WHERE station_id = " + str(i) + ";"

        
        # sql = 'UPDATE underground_stations SET url = ' + "'" + good_urls.at[i, 'url'] + "'" + " WHERE station_id = " + str(i) + ";"
        #print(sql)
        
        j = j+1
        cursor.execute(sql)

print(j)

30


In [8]:
good_urls.head()

,station,lines,url
index,,,
1,Aldgate,OM,https://www.google.com/maps/place/Aldgate/@51....
13,Barking,DH,https://www.google.com/maps/place/Barking/@51....
22,Blackfriars,OD,https://www.google.com/maps/place/Blackfriars/...
41,Chalfont & Latimer,M,https://www.google.com/maps/place/Chalfont+%26...
51,Clapham South,N,https://www.google.com/maps/place/Clapham+Sout...


In [9]:
cursor.execute("SELECT * FROM underground_stations")

result = cursor.fetchall()

test_df = pd.DataFrame(result)
test_df.columns = ['station_id', 'station', 'lines', 'local_authority', 'fare_zone', 'opened_on', 'annual_usage', 'area_served', 'url']
test_df.set_index('station_id', inplace=True)
test_df.head()

,station,lines,local_authority,fare_zone,opened_on,annual_usage,area_served,url
station_id,,,,,,,,
0,Acton Town,DP,Ealing,3,01-07-1879,4.93,Acton,https://www.google.com/maps?q=Acton+Town+Under...
2,Aldgate East,DH,Tower Hamlets,1,06-10-1884,10.23,Whitechapel,https://www.google.com/maps?q=Aldgate+East+Und...
3,Alperton,P,Brent,4,28-06-1903,2.30,Alperton,https://www.google.com/maps?q=Alperton+Undergr...
4,Amersham,M,Bucking­ham­shire,9,01-09-1892,1.66,Amersham,https://www.google.com/maps?q=Amersham+Undergr...
5,Angel,N,Islington,1,17-11-1901,12.37,Angel,https://www.google.com/maps?q=Angel+Undergroun...


In [27]:
stations_df[stations_df['station'] == 'Paddington']

,station,lines,local_authority,fare_zone,opened_on,annual_usage,area_served,url
station_id,,,,,,,,
173,Paddington,BOD,City of Westminster,1,01-10-1868,46.65,Paddington,https://www.google.com/maps?q=Paddington+Under...
174,Paddington,OH,City of Westminster,1,10-01-1863,46.65,Paddington,https://www.google.com/maps?q=Paddington+Under...


In [28]:
per_dict = {70: "https://www.google.com/maps/place/Edgware+Road/@51.5198669,-0.1693371,283m/data=!3m1!1e3!4m6!3m5!1s0x48761ab6aa336037:0x6d85207552a2db32!8m2!3d51.5202627!4d-0.1702314!16s%2Fg%2F1w8w9hlq!5m1!1e2?entry=ttu",
            71: "https://www.google.com/maps/place/Edgware+Road/@51.5198669,-0.1693371,283m/data=!3m1!1e3!4m6!3m5!1s0x48761ab42be4c56b:0xea21dc822667c51c!8m2!3d51.5198825!4d-0.1678336!16s%2Fm%2F025xwht!5m1!1e2?entry=ttu",
            173: "https://www.google.com/maps/place/Paddington/@51.5172907,-0.1828114,1105m/data=!3m1!1e3!4m6!3m5!1s0x48761ab2b05500a7:0x749d07ad72bbbe13!8m2!3d51.5160334!4d-0.1751595!16s%2Fg%2F1ptw36yt9!5m1!1e2?entry=ttu",
            174: "https://www.google.com/maps/place/Paddington/@51.5176667,-0.1802469,689m/data=!3m1!1e3!4m6!3m5!1s0x48761aadb931c0d3:0xffd8714ad4d3f207!8m2!3d51.5186053!4d-0.1781658!16s%2Fg%2F11ks459cq9!5m1!1e2?entry=ttu"}

In [35]:
# updating the paddington and edgware road station's urls
for key in per_dict.keys():
        if "'" in per_dict[key]:
                url = url.replace("'", "''")
                sql = 'UPDATE underground_stations SET url = ' + "'" + per_dict[key] + "'" + " WHERE station_id = " + str(key) + ";"
        else:
                sql = 'UPDATE underground_stations SET url = ' + "'" + per_dict[key] + "'" + " WHERE station_id = " + str(key) + ";"

        cursor.execute(sql)


In [ ]:
# Paddington
# 173 - circle and bake and district - https://www.google.com/maps/place/Paddington/@51.5172907,-0.1828114,1105m/data=!3m1!1e3!4m6!3m5!1s0x48761ab2b05500a7:0x749d07ad72bbbe13!8m2!3d51.5160334!4d-0.1751595!16s%2Fg%2F1ptw36yt9!5m1!1e2?entry=ttu
# 174 - hammer and circle - https://www.google.com/maps/place/Paddington/@51.5176667,-0.1802469,689m/data=!3m1!1e3!4m6!3m5!1s0x48761aadb931c0d3:0xffd8714ad4d3f207!8m2!3d51.5186053!4d-0.1781658!16s%2Fg%2F11ks459cq9!5m1!1e2?entry=ttu

# Edgware road
# 71 - Circle and Hammer - https://www.google.com/maps/place/Edgware+Road/@51.5198669,-0.1693371,283m/data=!3m1!1e3!4m6!3m5!1s0x48761ab42be4c56b:0xea21dc822667c51c!8m2!3d51.5198825!4d-0.1678336!16s%2Fm%2F025xwht!5m1!1e2?entry=ttu
# 70 - bake - https://www.google.com/maps/place/Edgware+Road/@51.5198669,-0.1693371,283m/data=!3m1!1e3!4m6!3m5!1s0x48761ab6aa336037:0x6d85207552a2db32!8m2!3d51.5202627!4d-0.1702314!16s%2Fg%2F1w8w9hlq!5m1!1e2?entry=ttu


In [36]:
# commiting the changes
conn.commit()

# Closing cursor
cursor.close()

# closing connection
conn.close()